# RAG desde PDF - ESPAÑOL

Este RAG recibe tres pdfs sobre animales y responde a unas preguntas

## 1. Código

In [1]:
# Introducción documento "externo"
import PyPDF2
pdf_names=[ "arboles", "flores", "plantas"]

pdf_paths=[]
for name in pdf_names:
    pdf_paths.append('pdfs/'+name+'.pdf')
text=""
for pdf in pdf_paths:
    with open(pdf, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text = text + page.extract_text() 

In [2]:
# Split del texto recibido
import langchain
from langchain.text_splitter  import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",  
    chunk_size=400,  
    chunk_overlap=50
)

splits = text_splitter.split_text(text)

In [3]:
#Vectorizar
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv
import os 

load_dotenv()

MONGODB_ATLAS_CLUSTER_URI = api_key = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
DB_NAME = os.getenv("DB_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv("ATLAS_VECTOR_SEARCH_INDEX_NAME")

client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

for split in splits:
    embedding = embeddings.embed_documents(split) 
    document = Document(
        page_content=split,  
        embedding=embedding 
    )

    vector_store.add_documents([document])
try:
    vector_store.create_vector_search_index(dimensions=768)
except Exception as e:
    print("Index has already been created")



retriever = vector_store.as_retriever()

/home/bigdata/miniconda3/envs/blalba/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index has already been created


In [4]:
# Ollama 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Responde a la pregunta concisamente y en castellano en base a este contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)


## 2. Preguntas

In [5]:
chain.invoke("¿Dime cómo se puede reproducir una planta?")

'Una planta puede reproducirse a través de un proceso llamado "germenación", donde el tallo de una planta madre crece hacia la superficie y forma raíces propias. También pueden reproducirse las plantas mediante división del estolón, que es la parte del tallo subterráneo que conecta la planta con su progenitor.'

In [6]:
chain.invoke("¿Cómo es la estructura de un árbol?")

'La estructura de un árbol se compone de varias partes, incluyendo las hojas, flores y frutos. Las hojas permiten la fotosíntesis, mientras que las flores y los frutos son responsables de la reproducción del árbol a través de la dispersión de semillas.'

In [7]:
chain.invoke("¿Cuál es la función del tronco y las ramas?")

'La función del tronco y las ramas es unir el árbol con las hojas y los frutos, proporcionando estructura y soporte. El tronco sirve como base del árbol y puede estar sin ramas, mientras que las ramas se extienden hacia afuera y contienen las hojas y los frutos.'

In [8]:
chain.invoke("Háblame de la clasificación de las flores")

'Las flores se clasifican según varios criterios:\n\n- **Según el número de carpelos**: Unicarpelar (solo un ovario) o Pluricarpelar (más de un carpelo, que puede estar unido o separado).\n\n- **Según la forma de las inflorescencias**: Racimo (varias flores que se insertan a lo largo de un eje) o Inflorescencia terminal (una flor central rodeada por muchas otras).\n\n- **Según la presencia de órganos sexuales**: \n  - Flor hermafrodita: tiene órganos masculinos y femeninos.\n  - Unisexual masculina: solo tiene estambres.\n  - Unisexual femenina: solo tiene carpelos.'

In [9]:
chain.invoke("¿Un árbol emite nitrógeno a la atmósfera?")

'No, no se menciona que un árbol emita nitrógeno a la atmósfera en este contexto.'